<a href="https://colab.research.google.com/github/kazikamil/backend_track/blob/main/tracking_model_1_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab Notebook Documentation

This notebook processes and merges data from three different sources related to a race event: lap analysis, weather conditions, and telemetry data. The goal is to combine these datasets, detect incidents, and calculate performance metrics like 'loss per lap'.

### 1. Data Loading and Initial Setup

This cell mounts Google Drive to access the data files and defines the directory where they are located. This is a prerequisite for loading the CSV files.

In [ ]:
# Cell 2 — Monter Google Drive (si les données sont dans MyDrive)
from google.colab import drive
drive.mount('/content/drive')

# Chemin vers le dossier contenant tp4_data (modifie si besoin)
data_dir = '/content/drive/MyDrive/tracking'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2. Loading Race Data

Here, we load the three primary CSV files into pandas DataFrames: `df1` (lap analysis), `df2` (weather data), and `df3` (telemetry data). `df3` is loaded in chunks due to its potentially large size to handle memory efficiently.

In [ ]:
import pandas as pd
import os

csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
print("Fichiers trouvés :", csv_files)

# Lire les deux fichiers CSV dans deux DataFrames
df1 = pd.read_csv('/content/drive/MyDrive/tracking/23_AnalysisEnduranceWithSections_Race 1_Anonymized.CSV', sep=';')
df2 = pd.read_csv("/content/drive/MyDrive/tracking/26_Weather_Race 1_Anonymized.CSV", sep=';')
#df3 = pd.read_csv("/content/drive/MyDrive/tracking/sonoma_telemetry_R1.csv.gz", compression='gzip')
#df3 = pd.read_csv("/content/drive/MyDrive/tracking/R2_road_america_telemetry_data.csv", sep=',')

#cols = pd.read_csv("/content/drive/MyDrive/tracking/sonoma_telemetry_R1.csv.gz", compression='gzip',nrows=5)
#print(cols.tolist())
chunksize = 100000  # nombre de lignes par bloc
results = []

for chunk in pd.read_csv("/content/drive/MyDrive/tracking/sonoma_telemetry_R1.csv.gz", compression='gzip' ,chunksize=chunksize):
  chunk = chunk[["lap", "vehicle_number", "telemetry_name","telemetry_value"]]
#  chunk = chunk[["lap", "vehicle_id", "value"]]

  results.append(chunk)

df3 = pd.concat(results)
    # Exemple : ne garder que certaines colonnes et filtrer

# Afficher un aperçu
print("\n--- Aperçu du premier fichier ---")
print(df1.head())

print("\n--- Aperçu du deuxième fichier ---")
print(df2.head())

print("\n--- Aperçu du troisième  fichier ---")
print(df3.head())

Fichiers trouvés : []

--- Aperçu du premier fichier ---
   NUMBER   DRIVER_NUMBER   LAP_NUMBER  LAP_TIME   LAP_IMPROVEMENT  \
0       3               1            1  2:44.364                 0   
1       3               1            2  2:01.931                 0   
2       3               1            3  1:56.717                 0   
3       3               1            4  1:56.554                 0   
4       3               1            5  2:03.062                 0   

   CROSSING_FINISH_LINE_IN_PIT        S1   S1_IMPROVEMENT      S2  \
0                          NaN  1:02.814                0  50.444   
1                          NaN    42.808                0  39.608   
2                          NaN    39.854                0  38.183   
3                          NaN    39.428                0  37.968   
4                          NaN    39.748                0  41.984   

    S2_IMPROVEMENT  ... INT-1_time  INT-1_elapsed  IM2a_time IM2a_elapsed  \
0                0  ...       

### 3. Cleaning `df1` Column Names

This cell standardizes the column names in `df1` by stripping leading/trailing whitespace and converting them to lowercase. This ensures consistency and easier manipulation of the DataFrame.

In [ ]:
print(df1.columns.tolist())
df1.columns = df1.columns.str.strip().str.lower()
print(df1.columns.tolist())


['NUMBER', ' DRIVER_NUMBER', ' LAP_NUMBER', ' LAP_TIME', ' LAP_IMPROVEMENT', ' CROSSING_FINISH_LINE_IN_PIT', ' S1', ' S1_IMPROVEMENT', ' S2', ' S2_IMPROVEMENT', ' S3', ' S3_IMPROVEMENT', ' KPH', ' ELAPSED', ' HOUR', 'S1_LARGE', 'S2_LARGE', 'S3_LARGE', 'TOP_SPEED', 'PIT_TIME', 'CLASS', 'GROUP', 'MANUFACTURER', 'FLAG_AT_FL', 'S1_SECONDS', 'S2_SECONDS', 'S3_SECONDS', 'IM1a_time', 'IM1a_elapsed', 'INT-1_time', 'INT-1_elapsed', 'IM2a_time', 'IM2a_elapsed', 'INT-2_time', 'INT-2_elapsed', 'IM3a_time', 'IM3a_elapsed', 'FL_time', 'FL_elapsed']
['number', 'driver_number', 'lap_number', 'lap_time', 'lap_improvement', 'crossing_finish_line_in_pit', 's1', 's1_improvement', 's2', 's2_improvement', 's3', 's3_improvement', 'kph', 'elapsed', 'hour', 's1_large', 's2_large', 's3_large', 'top_speed', 'pit_time', 'class', 'group', 'manufacturer', 'flag_at_fl', 's1_seconds', 's2_seconds', 's3_seconds', 'im1a_time', 'im1a_elapsed', 'int-1_time', 'int-1_elapsed', 'im2a_time', 'im2a_elapsed', 'int-2_time', 'in

### 4. Previewing `df1` Lap Data

This cell displays the first few rows of selected key columns from `df1`, providing a quick overview of the lap-specific data, including lap time, lap number, and driver identification.

In [ ]:
df1[['lap_time','lap_number','number','driver_number']].head()


,lap_time,lap_number,number,driver_number
0,2:44.364,1,3,1
1,2:01.931,2,3,1
2,1:56.717,3,3,1
3,1:56.554,4,3,1
4,2:03.062,5,3,1


### 5. Calculating Tyre Age

This complex operation calculates the 'tyre_age' for each lap. It sorts `df1` by vehicle and lap number, and then iterates through the laps to determine the number of laps completed since the last pit stop. The 'tyre_age' is reset to 0 if a pit stop ('pit_time') is recorded, simulating a tyre change.

In [ ]:
df1 = df1.sort_values(by=['number', 'lap_number']).copy()
df1['tyre_age'] = 0
print(df1.head())
for index, row in df1.iterrows():
    if pd.notna(row['pit_time']) and row['pit_time'] != '':
        df1.at[index, 'tyre_age'] = 0
        continue

    age = 0
    for index2, row2 in df1[df1['number'] == row['number']].iterrows():
        if index2 == index:
            break
        if pd.notna(row2['pit_time']) and row2['pit_time'] != '':
            age = 0  # on reset seulement la variable, pas la cellule du DataFrame
        else:
            age += 1

    df1.at[index, 'tyre_age'] = age


     number  driver_number  lap_number  lap_time  lap_improvement  \
207       2              1           1  2:42.005                0   
208       2              1           2  1:54.896                0   
209       2              1           3  1:54.105                0   
210       2              1           4  1:52.747                0   
211       2              1           5  2:06.068                0   

    crossing_finish_line_in_pit      s1  s1_improvement      s2  \
207                         NaN  50.838               0  51.959   
208                         NaN  40.002               0  37.136   
209                         NaN  38.212               0  38.157   
210                         NaN  37.817               0  37.347   
211                         NaN  37.724               0  40.347   

     s2_improvement  ... int-1_elapsed  im2a_time  im2a_elapsed int-2_time  \
207               0  ...        50.838     21.972      1:12.810     29.987   
208               0  ...  

### 6. Converting Time Columns to Datetime

This cell converts the 'hour' column in `df1` and 'TIME_UTC_STR' in `df2` to datetime objects. This is crucial for time-based operations, sorting, and merging later, ensuring proper temporal alignment.

In [ ]:
df1['hour'] = pd.to_datetime(df1['hour'], errors='coerce')
df2['TIME_UTC_STR'] = pd.to_datetime(df2['TIME_UTC_STR'], errors='coerce')


/tmp/ipython-input-1109015286.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['hour'] = pd.to_datetime(df1['hour'], errors='coerce')
/tmp/ipython-input-1109015286.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['TIME_UTC_STR'] = pd.to_datetime(df2['TIME_UTC_STR'], errors='coerce')


### 7. Initial Merge of Lap and Weather Data

This cell performs an initial merge between `df1` (lap data) and `df_weather` (which is `df2` sorted) using `pd.merge_asof`. It aligns laps with the nearest backward weather observation within a 5-minute tolerance, based on the 'hour' and 'TIME_UTC_STR' columns. This provides an initial association of weather conditions with laps.

In [ ]:
df1 = df1.sort_values('hour')
df_weather = df2.sort_values('TIME_UTC_STR')

df_merged = pd.merge_asof(
    df1,
    df_weather,
    left_on='hour',
    right_on='TIME_UTC_STR',
    direction='backward',  # ou 'nearest' selon ton besoin
    tolerance=pd.Timedelta('5min')  # tolérance de 5 min max
)


### 8. Calculating Elapsed Seconds for `df1`

This cell calculates 'elapsed_seconds' for `df1` relative to the start of the first lap. This helps in aligning time series data by providing a continuous time index for lap events.

In [ ]:
df1 = df1.sort_values('hour').copy()
df1['elapsed_seconds'] = (df1['hour'] - df1['hour'].iloc[0]).dt.total_seconds()


### 9. Preparing `df2` Time Data

This cell ensures `df2['TIME_UTC_SECONDS']` is numeric by converting it and sorts `df2` by this column. This preparation is essential for an accurate and efficient time-based merge with other DataFrames.

In [ ]:
df2['TIME_UTC_SECONDS'] = pd.to_numeric(df2['TIME_UTC_SECONDS'], errors='coerce')
df2 = df2.sort_values('TIME_UTC_SECONDS')

### 10. Ensuring Numeric Time Columns

This cell explicitly converts 'elapsed_seconds' in `df1` and 'TIME_UTC_SECONDS' in `df2` to float data types. This prevents potential issues during numeric calculations and merges, ensuring data type consistency.

In [ ]:
df1['elapsed_seconds'] = df1['elapsed_seconds'].astype(float)
df2['TIME_UTC_SECONDS'] = df2['TIME_UTC_SECONDS'].astype(float)


### 11. Synchronizing `df1` Time with UTC

This cell calculates a `TIME_UTC_SECOND` for `df1` by adding the 'elapsed_seconds' to the `race_start` time derived from `df2`. This step aims to align the lap event times with the absolute UTC timeline of the weather data, making them directly comparable.

In [ ]:
race_start = df2['TIME_UTC_SECONDS'].min()  # ou une valeur connue
df1['TIME_UTC_SECOND'] = df1['elapsed_seconds'] + race_start


### 12. Previewing Synchronized Time in `df1`

This cell shows the first few rows of the newly created `TIME_UTC_SECOND` alongside other key lap information from `df1`, allowing for a quick verification of the time synchronization.

In [ ]:
df1[['TIME_UTC_SECOND','lap_time','lap_number','number']].head()

,TIME_UTC_SECOND,lap_time,lap_number,number
302,1.743282e+09,2:41.685,1,46
92,1.743282e+09,2:41.772,1,13
408,1.743282e+09,2:41.856,1,55
485,1.743282e+09,2:41.949,1,72
207,1.743282e+09,2:42.005,1,2


### 13. Advanced Merge of Lap and Weather Data

This cell performs a more refined `pd.merge_asof` between `df1` and `df2` using the synchronized `TIME_UTC_SECOND` and `TIME_UTC_SECONDS` columns, with a 'nearest' direction and a 2-second tolerance. This ensures accurate weather data is associated with each lap, even if timestamps don't exactly match.

In [ ]:
df_merged = pd.merge_asof(
    df1.sort_values('TIME_UTC_SECOND'),
    df2.sort_values('TIME_UTC_SECONDS'),
    left_on='TIME_UTC_SECOND',
    right_on='TIME_UTC_SECONDS',
    direction='nearest',
    tolerance=2
)


### 14. Previewing Merged Lap and Weather Data

This cell displays key columns from the `df_merged` DataFrame, allowing a check of the successful integration of lap and weather information, including `AIR_TEMP` and `TRACK_TEMP`.

In [ ]:
df_merged[['number','hour','lap_time','lap_number','TIME_UTC_SECOND','TIME_UTC_SECONDS','AIR_TEMP','TRACK_TEMP']].head()


,number,hour,lap_time,lap_number,TIME_UTC_SECOND,TIME_UTC_SECONDS,AIR_TEMP,TRACK_TEMP
0,46,2025-11-24 14:08:30.508,2:41.685,1,1.743282e+09,1.743282e+09,15.8,32.2
1,13,2025-11-24 14:08:30.595,2:41.772,1,1.743282e+09,1.743282e+09,15.8,32.2
2,55,2025-11-24 14:08:30.679,2:41.856,1,1.743282e+09,1.743282e+09,15.8,32.2
3,72,2025-11-24 14:08:30.772,2:41.949,1,1.743282e+09,1.743282e+09,15.8,32.2
4,2,2025-11-24 14:08:30.828,2:42.005,1,1.743282e+09,1.743282e+09,15.8,32.2


### 15. Calculating Lap Time in Seconds and Start UTC

This cell first converts the string 'lap_time' into 'lap_time_seconds' (float) for numerical computations. Then, it calculates 'START_UTC_SECONDS' for each lap by subtracting the lap duration (`lap_time_seconds`) from the `TIME_UTC_SECOND` (which represents the end of the lap).

In [ ]:
df_merged['lap_time_seconds'] = (
    df_merged['lap_time']
    .str.split(':')
    .apply(lambda x: int(x[0]) * 60 + float(x[1]))
)

# Calcul du temps de début en secondes UTC
df_merged['TIME_UTC_SECONDS'] = pd.to_numeric(df_merged['TIME_UTC_SECONDS'], errors='coerce')

# 2️⃣ Calculer START_UTC_SECONDS
df_merged['START_UTC_SECONDS'] = df_merged['TIME_UTC_SECOND'] - df_merged['lap_time_seconds']



### 16. Previewing Calculated Lap Start Times

This cell displays the newly calculated 'lap_time_seconds' and 'START_UTC_SECONDS' alongside other relevant columns from `df_merged`, verifying the accuracy of the temporal calculations.

In [ ]:
df_merged[['number','lap_time', 'lap_time_seconds', 'START_UTC_SECONDS', 'TIME_UTC_SECONDS']].head()



,number,lap_time,lap_time_seconds,START_UTC_SECONDS,TIME_UTC_SECONDS
0,46,2:41.685,161.685,1.743282e+09,1.743282e+09
1,13,2:41.772,161.772,1.743282e+09,1.743282e+09
2,55,2:41.856,161.856,1.743282e+09,1.743282e+09
3,72,2:41.949,161.949,1.743282e+09,1.743282e+09
4,2,2:42.005,162.005,1.743282e+09,1.743282e+09


### 17. Verifying Time Range Consistency

This cell prints the minimum and maximum values for 'elapsed_seconds' in `df1` and 'TIME_UTC_SECONDS' in `df2`. This is a crucial check to ensure that the time ranges of both datasets overlap appropriately for accurate merging.

In [ ]:
print("df1 elapsed_seconds :", df1['elapsed_seconds'].min(), "→", df1['elapsed_seconds'].max())
print("df2 TIME_UTC_SECONDS :", df2['TIME_UTC_SECONDS'].min(), "→", df2['TIME_UTC_SECONDS'].max())


df1 elapsed_seconds : 0.0 → 2744.755
df2 TIME_UTC_SECONDS : 1743282362.0 → 1743285120.0


### 18. Ensuring 'vehicle_number' in Telemetry Data

This cell checks if `df3` (telemetry data) has a 'vehicle_number' column. If it's missing, it attempts to extract it from 'vehicle_id' (assuming a specific format, e.g., 'car-X' or 'vehicle-X') to ensure consistency for merging with other DataFrames.

In [ ]:
if "vehicle_number" not in df3.columns:
    df3["vehicle_number"] = df3["vehicle_id"].str.split("-").str[-1].astype(int)

### 19. Processing Raw Telemetry Data (if needed)

This cell is designed to process `df3` if the telemetry data is stored in a JSON-like string format within a 'value' column. It converts these strings to Python objects, explodes the list of dictionaries into separate rows, and then extracts 'telemetry_name' and 'telemetry_value' into distinct columns. This step normalizes the telemetry data from a nested structure into a more usable tabular format.

In [ ]:
import pandas as pd
import ast  # pour convertir les chaînes JSON-like en objets Python

if "telemetry_name" not in df3.columns:
 df3 = df3[["lap", "vehicle_number", "value"]]
# --- Exemple : ton DataFrame brut ---
# df_raw = pd.read_csv("tes_donnees.csv")

# 1️⃣ Convertir la chaîne JSON en liste Python (si elle n'est pas déjà de type list)
 df3["value"] = df3["value"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 2️⃣ Exploser la liste en plusieurs lignes
 df_exploded = df3.explode("value", ignore_index=True)

# 3️⃣ Extraire "name" et "value" dans deux colonnes séparées
 df_exploded["telemetry_name"] = df_exploded["value"].apply(lambda d: d["name"])
 df_exploded["telemetry_value"] = df_exploded["value"].apply(lambda d: d["value"])

# 4️⃣ Supprimer l’ancienne colonne
 df_exploded = df_exploded.drop(columns=["value", "expire_at"], errors="ignore")

# 5️⃣ Ajouter/mapper les colonnes lap et vehicle_number (si elles viennent d’ailleurs)
# Exemple fictif :
# df_exploded["lap"] = df_raw["lap_id"]  # si tu as un id de tour ailleurs
# df_exploded["vehicle_number"] = ...

# 🔥 Tu peux maintenant passer df_exploded dans ton code :
 df3 = df_exploded.copy()

### 20. Incident Detection and Telemetry Aggregation

This comprehensive cell performs several critical steps for telemetry data:
1.  **Defines `detect_incident`**: A helper function to identify potential incidents based on high acceleration (e.g., `accx_can`, `accy_can` > 5g) or intense braking (e.g., `pbrake_f`, `pbrake_r` > 90%).
2.  **Calculates mean telemetry values**: Groups `df3` by lap and vehicle, then pivots the data to get the average of each `telemetry_name` for a given lap/vehicle combination.
3.  **Detects incidents on raw data**: Pivots `df3` again, but this time keeping *all* raw values in a list for each telemetry type. This allows checking if *any* single data point within a lap exceeds the predefined incident thresholds.
4.  **Applies incident detection**: The `incident_for_lap` function applies the incident logic to these raw lists, flagging a lap as an 'incident' (1) if any threshold is crossed.
5.  **Merges incidents with averages**: The `incident` flag is merged back into the averaged telemetry data (`pivoted`), creating a more complete `pivoted` DataFrame.

In [ ]:
import pandas as pd
import numpy as np

# --- 1️⃣ Définir les seuils d'incident ---
def detect_incident(row):
    if (
        abs(row["accx_can"]) > 5
        or abs(row["accy_can"]) > 5
        or row.get("pbrake_f", 0) > 90
        or row.get("pbrake_r", 0) > 90
    ):
        return 1
    return 0

# --- 2️⃣ Calcul des moyennes par lap ---
grouped = (
    df3.groupby(["lap", "vehicle_number", "telemetry_name"], as_index=False)
       ["telemetry_value"].mean()
)

pivoted = grouped.pivot_table(
    index=["lap", "vehicle_number"],
    columns="telemetry_name",
    values="telemetry_value"
).reset_index()

pivoted.columns.name = None

# --- 3️⃣ Détection des incidents sur données brutes ---
# on suppose que df3 contient telemetry_name / telemetry_value
df_incident = df3.pivot_table(
    index=["lap", "vehicle_number"],
    columns="telemetry_name",
    values="telemetry_value",
    aggfunc=list  # on garde toutes les valeurs pour un lap
).reset_index()

df_incident.columns.name = None

# --- 4️⃣ Application de la détection sur chaque tour ---
def incident_for_lap(row):
    # On récupère les listes de valeurs brutes
    accx_list = row.get("accx_can", [])
    accy_list = row.get("accy_can", [])
    pbf_list = row.get("pbrake_f", [])
    pbr_list = row.get("pbrake_r", [])

    # Incident = 1 si au moins une valeur dépasse un seuil
    if any(abs(x) > 5 for x in accx_list) or any(abs(y) > 5 for y in accy_list):
        return 1
    if any(x > 90 for x in pbf_list) or any(x > 90 for x in pbr_list):
        return 1
    return 0

df_incident["incident"] = df_incident.apply(incident_for_lap, axis=1)

# --- 5️⃣ Fusion des incidents avec les moyennes ---
df_final = pivoted.merge(
    df_incident[["lap", "vehicle_number", "incident"]],
    on=["lap", "vehicle_number"],
    how="left"
)

df_final["incident"] = df_final["incident"].fillna(0)

pivoted=df_final


### 21. Previewing Aggregated Telemetry Data with Incidents

This cell displays the head of the `pivoted` DataFrame, which now contains the mean telemetry values per lap and vehicle, along with the calculated `incident` flag. This allows for a quick review of the processed telemetry data and detected incidents.

In [ ]:
pivoted

,lap,vehicle_number,Steering_Angle,accx_can,accy_can,ath,gear,nmot,pbrake_f,pbrake_r,speed,incident
0,1,2,-3.061205,0.003125,-0.037095,19.446458,0.736086,1930.904170,1.361968,1.389854,33.359870,0
1,1,3,-1.590487,0.010776,-0.083173,46.965977,2.304814,5017.151163,4.899311,4.958814,95.579360,1
2,1,5,-10.830052,0.003955,-0.044671,27.159748,1.718694,3362.364840,6.111437,6.148751,62.350572,0
3,1,7,-2.056163,0.010714,-0.036429,24.686442,0.894060,2225.364912,2.204564,2.236997,39.437792,0
4,1,11,-4.270889,0.004833,-0.065542,38.881931,2.490739,4880.587465,3.004354,3.036200,95.868372,1
...,...,...,...,...,...,...,...,...,...,...,...,...
677,26,51,0.541163,-0.000962,-0.009233,7.666446,0.966080,161.584856,0.137790,0.140497,1.371265,0
678,26,71,-10.146494,-0.027367,-0.001179,5.193699,1.073722,1483.027869,0.693010,0.721838,20.700557,0
679,26,78,12.766964,0.005564,0.001510,7.950281,0.936215,140.330302,0.858692,0.868780,1.605357,0
680,26,80,-7.430590,-0.026751,-0.005213,6.882477,0.925061,1649.707617,0.865216,0.891059,15.685430,0


### 22. Final Data Integration (Lap, Weather, and Telemetry)

This cell performs the ultimate merge, combining the `df_laps` (which is `df_merged` with vehicle number renamed and sorted) with the `df_telemetry` (which is `pivoted` and sorted). The merge is performed on `vehicle_number` and `lap_number` to create `df_final`, a comprehensive dataset containing all integrated lap, weather, and telemetry information for each lap.

In [ ]:
import pandas as pd

# Assure-toi que les colonnes temporelles sont bien en float
pivoted['lap'] = pd.to_numeric(pivoted['lap'], errors='coerce')
df_merged['START_UTC_SECONDS'] = pd.to_numeric(df_merged['START_UTC_SECONDS'], errors='coerce')
df_merged['TIME_UTC_SECOND'] = pd.to_numeric(df_merged['TIME_UTC_SECOND'], errors='coerce')

# On trie les deux DataFrames par temps
df_telemetry = pivoted.sort_values(['vehicle_number', 'lap'])
df_laps = df_merged.sort_values(['number', 'lap_number'])
df_laps = df_laps.rename(columns={'number': 'vehicle_number'})
df_laps = df_laps.dropna(subset=['START_UTC_SECONDS'])
df_laps = df_laps[df_laps['vehicle_number'].isin(df_telemetry['vehicle_number'])]


# Merge "asof" sur le début du tour
df_final = pd.merge(
    df_laps,
    df_telemetry,
    left_on=['vehicle_number', 'lap_number'],
    right_on=['vehicle_number', 'lap'],
    how='left'
)



#df_final = df_final[df_final['TIME_UTC_SEC'] <= df_final['TIME_UTC_SECOND']]



### 23. Reviewing Processed Telemetry (`df3`)

This cell displays the head of `df3` again. At this stage, `df3` should reflect the transformed structure after processing raw telemetry values into `telemetry_name` and `telemetry_value` columns, as well as the addition of `vehicle_number`.

In [ ]:
df3.head()

,lap,vehicle_number,telemetry_name,telemetry_value
0,1,2,accx_can,0.081
1,1,2,accy_can,-0.010
2,1,2,ath,24.900
3,1,2,pbrake_r,0.000
4,1,2,pbrake_f,0.000


### 24. Sorting the Final Dataset

This cell explicitly sorts the `df_final` DataFrame by `vehicle_number` and `lap_number` to ensure a consistent, logical, and ordered presentation of the combined data. This is useful for subsequent analysis and visualization.

In [ ]:
df_final = df_final.sort_values(['vehicle_number', 'lap_number'])

### 25. Checking Final Dataset Size

This cell displays the total number of rows in the `df_final` DataFrame, providing a quick check of the merged dataset's size and confirming that the merge operations were executed as expected.

In [ ]:
len(df_final)


669

### 26. Checking Lap Data Size

This cell displays the number of rows in the `df_laps` DataFrame, which represents the processed lap and weather data before the final merge with telemetry. This helps in verifying the number of laps that were successfully prepared for integration.

In [ ]:
len(df_laps)


669

### 27. Feature Engineering: Performance Metrics

This cell performs feature engineering to calculate key performance metrics:
-   The 'lap_time' column is converted to `timedelta` objects for accurate time difference calculations.
-   'best_lap_time' is determined for each `vehicle_number` by finding the minimum lap time.
-   'loss_per_lap' is calculated as the total seconds difference between each lap's time and the vehicle's best lap time, indicating performance deviation and consistency.

In [ ]:
df_final["lap_time"] = pd.to_timedelta("0:" + df_final["lap_time"])
df_final["best_lap_time"] = df_final.groupby("vehicle_number")["lap_time"].transform("min")
df_final["loss_per_lap"] = (df_final["lap_time"] - df_final["best_lap_time"]).dt.total_seconds()


### 28. Previewing 'Loss Per Lap'

This cell displays the first few values of the newly calculated 'loss_per_lap' column. This allows for a quick inspection of this crucial performance metric, which quantifies how much slower each lap was compared to the best lap for that vehicle.

In [ ]:
df_final["loss_per_lap"].head()

,loss_per_lap
0,49.966
1,2.857
2,2.066
3,0.708
4,14.029


### 29. Exporting Final Processed Data

This cell saves the complete and processed `df_final` DataFrame to a CSV file named `SON_race_1.csv`. This file contains all integrated lap, weather, and telemetry data, along with engineered features like 'loss_per_lap' and 'incident' flags, making it ready for further analysis or machine learning tasks.

In [ ]:
df_final.to_csv("SON_race_1.csv", index=False)

### 30. Final Data Preview

This cell displays the head of the `df_final` DataFrame one last time. This final preview confirms its structure and content after all processing, merging, and feature engineering steps have been completed, ensuring the dataset is as expected.

In [ ]:
df_final.head()

,vehicle_number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,accy_can,ath,gear,nmot,pbrake_f,pbrake_r,speed,incident,best_lap_time,loss_per_lap
0,2,1,1,0 days 00:02:42.005000,0,NaN,50.838,0,51.959,0,...,-0.037095,19.446458,0.736086,1930.904170,1.361968,1.389854,33.359870,0.0,0 days 00:01:52.039000,49.966
1,2,1,2,0 days 00:01:54.896000,0,NaN,40.002,0,37.136,0,...,-0.065836,30.295301,2.247688,5061.421569,1.947915,1.977038,93.381779,1.0,0 days 00:01:52.039000,2.857
2,2,1,3,0 days 00:01:54.105000,0,NaN,38.212,0,38.157,0,...,-0.092041,72.288386,2.804456,6148.833013,7.067973,7.145891,127.830749,1.0,0 days 00:01:52.039000,2.066
3,2,1,4,0 days 00:01:52.747000,0,NaN,37.817,0,37.347,0,...,-0.074044,70.197535,2.826291,6185.702544,6.957686,7.043371,129.026341,1.0,0 days 00:01:52.039000,0.708
4,2,1,5,0 days 00:02:06.068000,0,NaN,37.724,0,40.347,0,...,-0.087153,70.926737,2.839984,6227.311155,7.724810,7.809984,130.112642,1.0,0 days 00:01:52.039000,14.029


## Conclusion

This notebook provides a robust pipeline for integrating diverse race data, performing essential preprocessing, and deriving key performance indicators and event detections. The resulting `SON_race_1.csv` dataset is ready for further analysis, visualization, or machine learning applications.